In [1]:
import json
import copy
import gc
import os
import re
from collections import defaultdict
from pathlib import Path

import torch
from torch import nn
import numpy as np
import pandas as pd
from spacy.lang.en import English
from transformers.tokenization_utils import PreTrainedTokenizerBase
from transformers.models.deberta_v2 import DebertaV2ForTokenClassification, DebertaV2TokenizerFast
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers.trainer_utils import EvalPrediction
from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict, concatenate_datasets
import wandb

2025-03-26 03:42:58.273396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-26 03:42:58.273525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-26 03:42:58.403021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Config & Parameters

In [2]:
# basic random seed
import os 
import random
import numpy as np 

DEFAULT_RANDOM_SEED = 29

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# torch random seed
import torch
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

In [3]:
TRAINING_MODEL_PATH = "microsoft/deberta-v3-large"
TRAINING_MAX_LENGTH = 512
EVAL_MAX_LENGTH = 512
CONF_THRESH = 0.6
LR = 2.5e-5
LR_SCHEDULER_TYPE = "linear"
NUM_EPOCHS = 30
BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
GRAD_ACCUMULATION_STEPS = 16 // BATCH_SIZE
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
AMP = True
FREEZE_EMBEDDING = False
FREEZE_LAYERS = 2
N_SPLITS = 4
NEGATIVE_RATIO = 0.3  # down sample ratio of negative samples in the training set
OUTPUT_DIR = "output"
Path(OUTPUT_DIR).mkdir(exist_ok=True)

In [4]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=AMP,
    learning_rate=LR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    report_to="none",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
)

## Dataset Preparation

In [5]:
with open("/kaggle/input/workshop-task-acl/SOMD2025-PhaseI/train_texts.txt", "r", encoding="utf-8") as file:
    text = file.read()

text_list = text.split('\n')

with open("/kaggle/input/workshop-task-acl/SOMD2025-PhaseI/train_entities.txt", "r", encoding="utf-8") as file:
    text = file.read()

labels_list = text.split('\n')

In [6]:
df = pd.DataFrame()
df['train_text'] = text_list
df['train_labels'] = labels_list

In [7]:
df['train_text_list'] = df['train_text'].str.split(' ')
df['train_labels_list'] = df['train_labels'].str.split(' ')

In [8]:
df['unique_labels'] = df['train_labels_list'].apply(lambda x: ' '.join(list(set(x))))

In [9]:
df['document'] = np.arange(len(df))

In [10]:
x = ' '.join(df['unique_labels'].values)

In [11]:
list(set(x.split(' ')))

['I-Extension',
 '',
 'B-Application',
 'B-License',
 'B-PlugIn',
 'B-Extension',
 'I-OperatingSystem',
 'I-URL',
 'I-PlugIn',
 'B-URL',
 'I-Developer',
 'B-OperatingSystem',
 'I-Application',
 'B-Abbreviation',
 'I-Release',
 'B-Citation',
 'B-ProgrammingEnvironment',
 'B-SoftwareCoreference',
 'B-Developer',
 'I-Version',
 'B-AlternativeName',
 'I-AlternativeName',
 'I-SoftwareCoreference',
 'B-Release',
 'B-Version',
 'I-License',
 'I-Citation',
 'I-ProgrammingEnvironment',
 'O']

In [12]:
all_labels = [
    'B-Extension','I-Extension','B-Application','I-Application','B-Abbreviation','B-Citation','I-Citation',
    'B-SoftwareCoreference','I-SoftwareCoreference','B-URL','I-URL','B-AlternativeName', 'I-AlternativeName',
    'B-OperatingSystem','I-OperatingSystem','B-Developer','I-Developer','O','B-License','I-License','B-PlugIn','I-PlugIn',
    'B-Release','I-Release','B-ProgrammingEnvironment','I-ProgrammingEnvironment','B-Version','I-Version']

id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

## Tokenization

In [13]:
class CustomTokenizer:
    def __init__(self, tokenizer: PreTrainedTokenizerBase, label2id: dict, max_length: int) -> None:
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length

    def __call__(self, example: dict) -> dict:
        # rebuild text from tokens
        text, labels, token_map = [], [], []

        for idx, (t, l) in enumerate(
            zip(example["tokens"], example["provided_labels"])
        ):
            text.append(t)
            labels.extend([l] * len(t))
            token_map.extend([idx]*len(t))

        text = "".join(text)
        labels = np.array(labels)

        # actual tokenization
        tokenized = self.tokenizer(
            "".join(text),
            return_offsets_mapping=True,
            truncation=True,
            max_length=self.max_length
        )

        token_labels = []

        for start_idx, end_idx in tokenized.offset_mapping:
            # CLS token
            if start_idx == 0 and end_idx == 0:
                token_labels.append(self.label2id["O"])
                continue

            # case when token starts with whitespace
            if text[start_idx].isspace():
                start_idx += 1

            token_labels.append(self.label2id[labels[start_idx]])

        length = len(tokenized.input_ids)

        return {**tokenized, "labels": token_labels, "length": length, "token_map": token_map}

## Instantiate the dataset

In [14]:
tokenizer = DebertaV2TokenizerFast.from_pretrained(TRAINING_MODEL_PATH)
train_encoder = CustomTokenizer(tokenizer=tokenizer, label2id=label2id, max_length=TRAINING_MAX_LENGTH)
eval_encoder = CustomTokenizer(tokenizer=tokenizer, label2id=label2id, max_length=EVAL_MAX_LENGTH)

ds = DatasetDict()

ds = Dataset.from_dict({
        "full_text": df['train_text'].values.tolist(),
        "tokens": df['train_text_list'].values.tolist(),
        "document": [str(x) for x in df['document'].values.tolist()],
        "provided_labels": df['train_labels_list'].values.tolist()})

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Metrics

In [15]:
def find_span(target: list[str], document: list[str]) -> list[list[int]]:
    idx = 0
    spans = []
    span = []

    for i, token in enumerate(document):
        if token != target[idx]:
            idx = 0
            span = []
            continue
        span.append(i)
        idx += 1
        if idx == len(target):
            spans.append(span)
            span = []
            idx = 0
            continue

    return spans


class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f1(self) -> float:
        beta = 1
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f1": self.f1}

In [16]:
class MetricsComputer:
    nlp = English()

    def __init__(self, eval_ds: Dataset, label2id: dict, conf_thresh: float = 0.9) -> None:
        self.ds = eval_ds.remove_columns("labels").rename_columns({"provided_labels": "labels"})
        self.gt_df = self.create_gt_df(self.ds)
        self.label2id = label2id
        self.confth = conf_thresh

    def __call__(self, eval_preds: EvalPrediction) -> dict:
        pred_df = self.create_pred_df(eval_preds.predictions)
        return self.compute_metrics_from_df(self.gt_df, pred_df)

    @staticmethod
    def create_gt_df(ds: Dataset):
        gt = []
        for row in ds:
            for token_idx, (token, label) in enumerate(zip(row["tokens"], row["labels"])):
                if label == "O":
                    continue
                gt.append(
                    {"document": row["document"], "token": token_idx, "label": label, "token_str": token}
                )
        gt_df = pd.DataFrame(gt)
        gt_df["row_id"] = gt_df.index

        return gt_df

    def create_pred_df(self, logits: np.ndarray) -> pd.DataFrame:
        """
        Note:
            Thresholing is doen on logits instead of softmax, which could find better models on LB.
        """
        prediction = logits
        o_index = self.label2id["O"]
        preds = prediction.argmax(-1)
        preds_without_o = prediction.copy()
        preds_without_o[:,:,o_index] = 0
        preds_without_o = preds_without_o.argmax(-1)
        o_preds = prediction[:,:,o_index]
        preds_final = np.where(o_preds < self.confth, preds_without_o , preds)

        pairs = set()
        processed = []

        # Iterate over document
        for p_doc, token_map, offsets, tokens, doc in zip(
            preds_final, self.ds["token_map"], self.ds["offset_mapping"], self.ds["tokens"], self.ds["document"]
        ):
            # Iterate over sequence
            for p_token, (start_idx, end_idx) in zip(p_doc, offsets):
                label_pred = id2label[p_token]

                if start_idx + end_idx == 0:
                    # [CLS] token i.e. BOS
                    continue

                if token_map[start_idx] == -1:
                    start_idx += 1

                # ignore "\n\n"
                while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                    start_idx += 1

                if start_idx >= len(token_map):
                    break

                token_id = token_map[start_idx]
                pair = (doc, token_id)

                # ignore "O", preds, phone number and  email
                # if label_pred in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM") or token_id == -1:
                #     continue
                if label_pred in ("O") or token_id == -1:
                    continue

                if pair in pairs:
                    continue

                processed.append(
                    {"document": doc, "token": token_id, "label": label_pred, "token_str": tokens[token_id]}
                )
                pairs.add(pair)

        # pred_df = pd.DataFrame(processed + self.emails + self.phone_nums)
        pred_df = pd.DataFrame(processed)
        pred_df["row_id"] = list(range(len(pred_df)))

        return pred_df

    def compute_metrics_from_df(self, gt_df, pred_df):
        """
        Compute the LB metric (lb) and other auxiliary metrics
        """

        references = {(row.document, row.token, row.label) for row in gt_df.itertuples()}
        predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

        score_per_type = defaultdict(PRFScore)
        references = set(references)

        for ex in predictions:
            pred_type = ex[-1] # (document, token, label)
            if pred_type != 'O':
                pred_type = pred_type[2:] # avoid B- and I- prefix

            if pred_type not in score_per_type:
                score_per_type[pred_type] = PRFScore()

            if ex in references:
                score_per_type[pred_type].tp += 1
                references.remove(ex)
            else:
                score_per_type[pred_type].fp += 1

        for doc, tok, ref_type in references:
            if ref_type != 'O':
                ref_type = ref_type[2:] # avoid B- and I- prefix

            if ref_type not in score_per_type:
                score_per_type[ref_type] = PRFScore()
            score_per_type[ref_type].fn += 1

        totals = PRFScore()

        for prf in score_per_type.values():
            totals += prf

        return {
            "precision": totals.precision,
            "recall": totals.recall,
            "f1": totals.f1,
            **{
                f"{v_k}-{k}": v_v
                for k in set([l[2:] for l in self.label2id.keys() if l!= 'O'])
                for v_k, v_v in score_per_type[k].to_dict().items()
            },
        }

## Model

In [17]:
class ModelInit:
    def __init__(
        self,
        checkpoint: str,
        id2label: dict,
        label2id: dict,
        freeze_embedding: bool,
        freeze_layers: int,
    ) -> None:
        self.model = DebertaV2ForTokenClassification.from_pretrained(
            checkpoint,
            num_labels=len(id2label),
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
        for param in self.model.deberta.embeddings.parameters():
            param.requires_grad = False if freeze_embedding else True
        for layer in self.model.deberta.encoder.layer[:freeze_layers]:
            for param in layer.parameters():
                param.requires_grad = False
        self.weight = copy.deepcopy(self.model.state_dict())

    def __call__(self) -> DebertaV2ForTokenClassification:
        self.model.load_state_dict(self.weight)
        return self.model

model_init = ModelInit(
    TRAINING_MODEL_PATH,
    id2label=id2label,
    label2id=label2id,
    freeze_embedding=FREEZE_EMBEDDING,
    freeze_layers=FREEZE_LAYERS,
)

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Split 
Split the original dataset into 4 folds according to `document % 4` <br>
Only uses the first 30% of negative samples in the training set but they are NOT excluded from the eval set to make sure cross-evalidation is done on the entire training dataset.

In [18]:
# split according to document id
folds = [
    (
        np.array([i for i, d in enumerate(ds["document"]) if int(d) % N_SPLITS != s]),
        np.array([i for i, d in enumerate(ds["document"]) if int(d) % N_SPLITS == s])
    )
    for s in range(N_SPLITS)
]

negative_idxs = [i for i, labels in enumerate(ds["provided_labels"]) if not any(np.array(labels) != "O")]
exclude_indices = negative_idxs[int(len(negative_idxs) * NEGATIVE_RATIO):]

## Train
Performs cross-validation and save the best checkpoint's metrics as json.

In [19]:
for fold_idx, (train_idx, eval_idx) in enumerate(folds):
    if fold_idx==2:
        print('fold_idx: ', fold_idx)
        args.run_name = f"fold-{fold_idx}"
        args.output_dir = os.path.join(OUTPUT_DIR, f"fold_{fold_idx}")
        original_ds = ds.select([i for i in train_idx if i not in exclude_indices])
        train_ds = original_ds
        train_ds = train_ds.map(train_encoder, num_proc=os.cpu_count())
        eval_ds = ds.select(eval_idx)
        eval_ds = eval_ds.map(eval_encoder, num_proc=os.cpu_count())
        trainer = Trainer(
            args=args,
            model_init=model_init,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            tokenizer=tokenizer,
            compute_metrics=MetricsComputer(eval_ds=eval_ds, label2id=label2id),
            data_collator=DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16),
        )
        trainer.train()
        eval_res = trainer.evaluate(eval_dataset=eval_ds)
        with open(os.path.join(args.output_dir, "eval_result.json"), "w") as f:
            json.dump(eval_res, f)
        del trainer
        gc.collect()
        torch.cuda.empty_cache()
    else:
        pass

fold_idx:  2
     

#0:   0%|          | 0/203 [00:00<?, ?ex/s]

#1:   0%|          | 0/203 [00:00<?, ?ex/s]

#2:   0%|          | 0/202 [00:00<?, ?ex/s]

#3:   0%|          | 0/202 [00:00<?, ?ex/s]

...

#1:   0%|          | 0/72 [00:00<?, ?ex/s]

#2:   0%|          | 0/72 [00:00<?, ?ex/s]

#3:   0%|          | 0/71 [00:00<?, ?ex/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,P-extension,R-extension,F1-extension,P-alternativename,R-alternativename,F1-alternativename,P-operatingsystem,R-operatingsystem,F1-operatingsystem,P-url,R-url,F1-url,P-license,R-license,F1-license,P-release,R-release,F1-release,P-softwarecoreference,R-softwarecoreference,F1-softwarecoreference,P-citation,R-citation,F1-citation,P-abbreviation,R-abbreviation,F1-abbreviation,P-application,R-application,F1-application,P-version,R-version,F1-version,P-programmingenvironment,R-programmingenvironment,F1-programmingenvironment,P-plugin,R-plugin,F1-plugin,P-developer,R-developer,F1-developer
1,No log,0.740596,0.614308,0.282143,0.386686,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401361,0.967213,0.567308,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.644231,0.329238,0.435772,0.701389,0.918182,0.795276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,No log,0.266787,0.763424,0.700714,0.730726,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.212121,0.350000,0.842857,0.967213,0.900763,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.873786,0.720000,0.789474,0.000000,0.000000,0.000000,0.630798,0.835381,0.718816,0.827451,0.959091,0.888421,0.642857,0.130435,0.216867,0.000000,0.000000,0.000000,0.892256,0.898305,0.895270
3,No log,0.181575,0.837739,0.815000,0.826213,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.906250,0.878788,0.892308,0.852941,0.950820,0.899225,0.888889,0.888889,0.888889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.864865,0.768000,0.813559,0.909091,0.909091,0.909091,0.792494,0.882064,0.834884,0.862348,0.968182,0.912206,0.797101,0.797101,0.797101,0.666667,0.285714,0.400000,0.891447,0.918644,0.904841
4,No log,0.160852,0.845619,0.868571,0.856942,1.000000,0.133333,0.235294,0.250000,0.166667,0.200000,0.909091,0.909091,0.909091,0.893939,0.967213,0.929134,0.866667,0.962963,0.912281,0.760000,0.612903,0.678571,0.000000,0.000000,0.000000,0.800000,0.768000,0.783673,0.625000,0.909091,0.740741,0.822727,0.889435,0.854782,0.972727,0.972727,0.972727,0.878788,0.840580,0.859259,0.587156,0.703297,0.640000,0.904290,0.928814,0.916388
5,No log,0.161022,0.897929,0.867143,0.882267,1.000000,0.466667,0.636364,1.000000,0.500000,0.666667,0.970588,1.000000,0.985075,0.904762,0.934426,0.919355,0.870968,1.000000,0.931034,0.833333,0.645161,0.727273,0.000000,0.000000,0.000000,0.869159,0.744000,0.801724,0.875000,0.636364,0.736842,0.881068,0.891892,0.886447,0.941964,0.959091,0.950450,0.936508,0.855072,0.893939,0.737500,0.648352,0.690058,0.928328,0.922034,0.925170
6,No log,0.181632,0.877266,0.898571,0.887791,1.000000,0.466667,0.636364,0.571429,0.666667,0.615385,1.000000,0.969697,0.984615,0.893939,0.967213,0.929134,0.870968,1.000000,0.931034,0.814815,0.709677,0.758621,0.000000,0.000000,0.000000,0.842975,0.816000,0.829268,0.900000,0.818182,0.857143,0.880952,0.909091,0.894800,0.938326,0.968182,0.953020,0.878378,0.942029,0.909091,0.741573,0.725275,0.733333,0.879747,0.942373,0.909984
7,No log,0.142640,0.871636,0.902143,0.886627,1.000000,0.466667,0.636364,0.500000,0.750000,0.600000,0.885714,0.939394,0.911765,0.893939,0.967213,0.929134,0.870968,1.000000,0.931034,0.833333,0.645161,0.727273,0.000000,0.000000,0.000000,0.862069,0.800000,0.829876,0.900000,0.818182,0.857143,0.877934,0.918919,0.897959,0.955556,0.977273,0.966292,0.955882,0.942029,0.948905,0.613445,0.802198,0.695238,0.901316,0.928814,0.914858
8,No log,0.157074,0.890071,0.896429,0.893238,1.000000,0.466667,0.636364,0.500000,0.750000,0.600000,1.000000,0.969697,0.984615,0.893939,0.967213,0.929134,0.870968,1.000000,0.931034,0.923077,0.774194,0.842105,1.000000,0.333333,0.500000,0.831933,0.792000,0.811475,0.666667,0.909091,0.769231,0.916667,0.864865,0.890013,0.968037,0.963636,0.965831,0.942029,0.942029,0.942029,0.593750,0.835165,0.6940